## Imports

In [ ]:
# Import the usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Import Olist data
from olist.data import Olist
from olist.order import Order

# Olist's Net Promoter Score (NPS) 🔥

The **Net Promoter Score (NPS)** of a service answers the following question:

> How likely is it that you would recommend our company/product/service to a friend or colleague?

For a service rated between 1 and 5 stars, like Olist, we can **classify customers into three categories** based on their answers:
- ✅ **Promoters**: customers who answered  with a score of 5
- 😴 **Passive**: customers who answered with a score of 4 
- 😡 **Detractors**: customers who answered with a score between 1 and 3 (inclusive)

<br>

👉 NPS is computed by subtracting the percentage of customers who are **detractors** from the percentage of customers who are **promoters**.

> NPS  
= % Promoters - % Detractors   
= (# Promoter - # Detractors) / # Reviews  
= (# 5 stars - # <4 stars) / # Reviews

## Computing the Overall NPS Score of Olist

In [ ]:
data = Olist().get_data()
orders = Order().get_training_data()

In [ ]:
orders.review_score

0        4
1        4
2        5
3        5
4        5
        ..
96356    5
96357    4
96358    5
96359    2
96360    5
Name: review_score, Length: 96353, dtype: int64

In [ ]:
type(orders.review_score)

pandas.core.series.Series

❓Create a function that converts `review_score` into `nps_class`. `nps_class` should be a **classification** depending on the `review_score`, so there are 3 possibilities:

- `review_score` is **5** 👉 `nps_class` is **1** (promoter)
- `review_score` is **4** 👉 `nps_class` is **0** (passive)
- `review_score` is **3** or less 👉 `nps_class` is **-1** (detractor)

In [ ]:
def promoter_score(review_score):
    # $CHALLENGIFY_BEGIN
    score = 0

    if review_score == 5:
        score = 1
    if review_score < 4:
        score = -1

    return score

orders.review_score.map(promoter_score)
    # $CHALLENGIFY_END

0        0
1        0
2        1
3        1
4        1
        ..
96356    1
96357    0
96358    1
96359   -1
96360    1
Name: review_score, Length: 96353, dtype: int64

💡Let's try to rewrite this function into a single line of code that achieves the same result 😏

There are **several** ways to do it! Let's look at some of them, then we can compare their execution times to that of our function to see which one is more efficient ⏱️

Two general principles when it comes to programming/coding are:
- `KISS`: **K**eep **I**t **S**imple and **S**mart
- `DRY`: **D**on't **R**epeat **Y**ourself 😉

<details>
    <summary>💡Hint</summary>

Use the following methods and use `%time` to compare their execution times:
- `.apply()` with the function you wrote above
- `.map()` or `.apply()` with a `lambda` function
- `.loc[]` with boolean indexing
- `np.select()` with matching conditions

</details>    

In [ ]:
%%time
# Using .apply() with our function
orders['promoter_class'] = orders.review_score.apply(promoter_score)

CPU times: user 38.7 ms, sys: 2.88 ms, total: 41.5 ms
Wall time: 40.7 ms


In [ ]:
%%time
# Using .map() with our function
orders['promoter_class'] = orders.review_score.map(promoter_score)

CPU times: user 35.9 ms, sys: 2.55 ms, total: 38.5 ms
Wall time: 37.9 ms


In [ ]:
%%time
# Using .map() with lambda + chained ternary if statement; not recommended because it's harder to read
orders['promoter_class'] = orders.review_score.map(lambda review_score: 1 if review_score == 5 else -1 if review_score < 4 else 0)

CPU times: user 37.4 ms, sys: 1.84 ms, total: 39.2 ms
Wall time: 39.3 ms


In [ ]:
%%time
# Using .map() with a dictionary
orders['promoter_class'] = orders.review_score.map({5: 1, 4: 0, 3: -1, 2: -1, 1: -1})

CPU times: user 3.14 ms, sys: 1.56 ms, total: 4.7 ms
Wall time: 2.58 ms


In [ ]:
%%time
# Using .loc[] with boolean indexing (takes 1.91µs)
# Setting conditions
promoter = orders['review_score'] == 5
passive = orders['review_score'] == 4
detractor = orders['review_score'] <= 3

orders.loc[promoter, 'promoter_class'] = 1
orders.loc[passive, 'promoter_class'] = 0
orders.loc[detractor, 'promoter_class'] = -1

CPU times: user 5.87 ms, sys: 2.37 ms, total: 8.25 ms
Wall time: 5.64 ms


In [ ]:
%%time
# Using np.select()
orders['promoter_class'] = np.select([orders.review_score == 5,orders.review_score == 4,orders.review_score < 4 ], [1, 0, -1])


CPU times: user 4.9 ms, sys: 4.2 ms, total: 9.11 ms
Wall time: 7.04 ms


In [ ]:
%%time
# Even more concisely with np.select()
orders['promoter_class'] = np.select([orders.review_score >= 4], [orders.review_score - 4], -1)

CPU times: user 3.26 ms, sys: 1.32 ms, total: 4.59 ms
Wall time: 2.68 ms


In [ ]:
orders.head()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value,promoter_class
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0,0,4,1,1,29.99,8.72,0
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered,0,0,4,1,1,118.70,22.76,0
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered,1,0,5,1,1,159.90,19.22,1
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered,1,0,5,1,1,45.00,27.20,1
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered,1,0,5,1,1,19.90,8.72,1


**A Note About `.apply()`**

Consider the following examples:

```python
df.apply(lambda col: col.max(), axis = 0)
df.apply(lambda row: row['A'] + row['B'], axis = 1)
```

These operations look similar because they both use `.apply()`, but one is much slower than the other. The data layout for Pandas DataFrames is **column-major** (read more [here](https://en.wikipedia.org/wiki/Row-_and_column-major_order)), which means that column-wise operations are always going to be faster than row-wise operations. The second example above uses `axis=1`, making it a row-wise operation, which would be more appropriate for **row-major** data layouts such as NumPy arrays.

For small amounts of data, this difference is irrelevant, but when you start working with huge datasets this will probably make a big difference. For big datasets, you're likely to notice that using `.loc[]`, `np.select()` or `np.apply_along_axis()` will run faster on Pandas DataFrames when applying a function on every row.

It's always good to understand how your data is stored before you access it!

👇 Now that you have the different promoter scores, you can compute `Olist's NPS`.

In [ ]:
nps = orders.review_score.map({5: 1, 4: 0, 3: -1, 2: -1, 1: -1}).sum() / orders.review_score.count()

In [ ]:
# Displaying as a percentage to 1 decimal place e.g "NPS score = 47.8%"
f'NPS Score: {nps * 100:.1f}%'

'NPS Score: 38.1%'

## NPS per Customer State

👇 Here is the part of Olist's DB schema that is relevant for this section, to help you have an overview of things.

<img src="https://wagon-public-datasets.s3-eu-west-1.amazonaws.com/04-Decision-Science/02-Statistical-Inference/olist_schema_for_nps.png" width=750>

### What is the average review score per state?

❓First, create the dataset required for computation

In [ ]:
# Practice "chaining" methods in pandas
# $CHALLENGIFY_BEGIN
merge = data['orders']\
.merge(data['order_reviews'], on='order_id')\
.merge(data['customers'], on='customer_id')

len(merge)
# $CHALLENGIFY_END

99224

In [ ]:
merge.nunique()

order_id                         98673
customer_id                      98673
order_status                         8
order_purchase_timestamp         98115
order_approved_at                90082
order_delivered_carrier_date     80451
order_delivered_customer_date    95022
order_estimated_delivery_date      459
review_id                        98410
review_score                         5
review_comment_title              4527
review_comment_message           36159
review_creation_date               636
review_answer_timestamp          98248
customer_unique_id               95380
customer_zip_code_prefix         14973
customer_city                     4117
customer_state                      27
dtype: int64

👉 Now, we can aggregate this dataset per  `customer_state` using any aggregation method of our choice :)

❓ Let's start with the average review score: compute the average `review_score` per `customer_state`.

*Hints:* try to tackle this question using three different methods:
- with `.mean()`
- then with `.apply()`
- and eventually the `.agg()`

In [ ]:
# Compute the average review_score per customer_state
merge.groupby(['customer_state'])['review_score'].mean().head()

customer_state
AC    4.049383
AL    3.751208
AM    4.183673
AP    4.194030
BA    3.860888
Name: review_score, dtype: float64

In [ ]:
# Use .apply() to do the same thing
merge.groupby(['customer_state'])['review_score'].apply(np.mean).head()

customer_state
AC    4.049383
AL    3.751208
AM    4.183673
AP    4.194030
BA    3.860888
Name: review_score, dtype: float64

In [ ]:
# Try with .agg(), it's much more flexible!
merge.groupby(['customer_state']).agg({'review_score': np.mean})\
.rename(columns={'review_score': 'mean_review_score'})\
.head()

,mean_review_score
customer_state,
AC,4.049383
AL,3.751208
AM,4.183673
AP,4.194030
BA,3.860888


🤩 `.agg()` is much more flexible than the other methods, push it further!

In [ ]:
merge.groupby(['customer_state']).agg({
    'review_score': [max, np.mean],
    'customer_zip_code_prefix': [pd.Series.nunique]
}).head()\
#.loc[:, ('review_score', 'max')]

review_score           customer_zip_code_prefix
                        max      mean                  nunique
customer_state                                                
AC                        5  4.049383                       20
AL                        5  3.751208                      126
AM                        5  4.183673                       55
AP                        5  4.194030                       18
BA                        5  3.860888                      734

### NPS per State

❓Now, it is time to create a 🔥 **custom aggregation function** to compute the `NPS per customer_state` directly.

1️⃣ Create your `nps` function

2️⃣ Try to debug it using the `breakpoint()` debugger within your function to understand clearly what objects you are manipulating

<br>

💡 *PS.:* always **cleanly** exit your debugger by typing `exit` when inside the debugging session, otherwise you will have to restart your Notebook!

In [ ]:
def nps(series):
    # breakpoint()
    return series.map(promoter_score).sum() / series.count()

👉 Now, use your `nps` function to compute the `NPS per customer_state`.

In [ ]:
merge.groupby(['customer_state']).agg({
    'review_score': nps,
}).head()

,review_score
customer_state,
AC,0.296296
AL,0.166667
AM,0.414966
AP,0.313433
BA,0.199881


Again, instead of using this function, try to do the same task in one line of code, remember the `KISS` principle? 😉

In [ ]:
# Challenge: Try a one-liner code!
merge.groupby(['customer_state'])['review_score'].apply(nps)

customer_state
AC    0.296296
AL    0.166667
AM    0.414966
AP    0.313433
BA    0.199881
CE    0.207675
DF    0.333799
ES    0.311012
GO    0.305830
MA    0.156836
MG    0.372989
MS    0.374309
MT    0.343300
PA    0.205579
PB    0.306968
PE    0.312272
PI    0.250509
PR    0.398769
RJ    0.253349
RN    0.358921
RO    0.301587
RR    0.043478
RS    0.368959
SC    0.332597
SE    0.206304
SP    0.397481
TO    0.326165
Name: review_score, dtype: float64

# Cheat Sheet for `map`, `apply`, `applymap` and `groupby`

```python
# MAP (for Series)
series.map(function) 
series.map({mapping dict})

# APPLY (for DataFrame)
df.apply(lambda col: col.max(), axis = 0)     # default axis
df.apply(lambda row: row[‘A’] + row[‘B’], axis = 1)

df.applymap(my_funct_for_indiv_elements)
df.applymap(lambda x: '%.2f' % x)
```

```python
## GROUPBY
group = df.groupby('col_A')

group.mean()
group.apply(np.mean)
group.agg({
    col_A: ['mean', np.sum],
    col_B: my_custom_sum,
    col_B: lambda s: my_custom_sum(s)
})

group.apply(custom_mean_function)
```

[Introduction to Pandas' `apply`, `applymap` and `map` - Towards Data Science](https://medium.com/towards-data-science/introduction-to-pandas-apply-applymap-and-map-5d3e044e93ff)